In [1]:
# Constructs Time Series Data for All Stocks
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import tushare as ts

from scipy.stats import rankdata

import seaborn as sns

# Pull All Trade Dates
trade_dates = pd.Series(data=[pd.Timestamp(date) for date in get_trading_dates('2001-01-01', '2018-12-31')], name='trade_date')

# year_start = 2001
year_start = 2012
year_end = 2018

# date_end_last_dt = max(trade_dates[trade_dates.dt.year == year_start-1])
date_start_dt = min(trade_dates[trade_dates.dt.year == year_start])
date_end_dt = max(trade_dates[trade_dates.dt.year == year_end])
# date_end_dt = date_start_dt+timedelta(days=1) # 2012-01-05
# date_end_last = date_end_last_dt.strftime('%Y-%m-%d')
date_start = date_start_dt.strftime('%Y-%m-%d')
date_end = date_end_dt.strftime('%Y-%m-%d')

# Construct Stock Population
stock_all = all_instruments(type="CS", country='cn', date=date_start_dt)
stock_list = stock_all['order_book_id'].tolist()
print("Population Check - Initial #: {}".format(stock_all.shape[0]))

Population Check - Initial #: 2320


In [2]:
price_data = get_price(stock_list, start_date=date_start, end_date=date_end, frequency='1d', 
                       fields=['close'], 
                       adjust_type='pre', skip_suspended=False, country='cn')
price_data.to_csv("cn_stock_price_{}_{}.csv".format(year_start,year_end)) # Download price data

In [3]:
price_data.head()

,300188.XSHE,600337.XSHG,600168.XSHG,002337.XSHE,600592.XSHG,000950.XSHE,600991.XSHG,002473.XSHE,600784.XSHG,600736.XSHG,...,600345.XSHG,600387.XSHG,000063.XSHE,002506.XSHE,300151.XSHE,002579.XSHE,000563.XSHE,000551.XSHE,002578.XSHE,000726.XSHE
2012-01-04,4.3401,3.1750,6.3693,2.8752,6.8314,4.0168,15.08,7.2898,5.0104,4.0135,...,9.4373,8.7711,13.3691,2.6406,2.0055,3.4037,2.3129,5.7181,1.8931,6.0165
2012-01-05,4.0743,2.8583,6.1697,2.6739,6.1737,3.9101,15.08,6.9505,4.5089,3.8534,...,8.8456,8.5560,13.2895,2.3920,1.8872,3.1452,2.2942,5.2525,1.7877,6.0005
2012-01-06,4.1173,2.6920,5.9511,2.7148,6.2207,4.1429,15.11,7.0765,4.6111,3.9570,...,8.9315,8.4289,12.9473,2.4624,1.9159,3.1724,2.2475,5.2787,1.7897,6.2645
2012-01-09,4.4041,2.7791,6.2172,2.8154,6.4744,4.2787,15.12,7.4449,4.8741,4.2114,...,9.3705,8.9765,13.2975,2.5991,2.0094,3.3194,2.3362,5.7050,1.8814,6.4965
2012-01-10,4.5124,2.9137,6.4549,2.9570,6.7845,4.4437,15.29,7.7357,5.1370,4.2962,...,9.7712,9.3285,13.8864,2.7565,2.1143,3.4255,2.4087,5.9410,2.0024,6.6565


In [4]:
trade_data = get_price(stock_list, start_date=date_start, end_date=date_end, frequency='1d', 
                       fields=['close', 'total_turnover', 'volume'], 
                       adjust_type='pre', skip_suspended=False, country='cn')

In [5]:
return_data = get_price_change_rate(stock_list, start_date=date_start, end_date=date_end)

In [6]:
turnover_data = get_turnover_rate(stock_list, date_start, date_end, fields=['week', 'month'])

In [9]:
instrument_info = instruments(stock_list)

In [25]:
instrument_info[0]

Instrument(industry_name='软件和信息技术服务业', sector_code_name='信息技术', abbrev_symbol='MYBK', listed_date='2011-03-16', exchange='XSHE', symbol='美亚柏科', industry_code='I65', round_lot=100.0, order_book_id='300188.XSHE', special_type='Normal', shenwan_industry_name='计算机', de_listed_date='0000-00-00', type='CS', sector_code='InformationTechnology', board_type='GEM', shenwan_industry_code='801750.INDX', status='Active')

In [27]:
# Download instrument information

bookId_list = []
exchange_list = []
abbrevSymbol_list = []
shenwanIndustryCode_list = []
shenwanIndustryName_list = []
industryCode_list = []
industryName_list = []
sectorCode_list = []
sectorName_list = []
for inst in instrument_info:
  bookId_list.append(inst.order_book_id)
  exchange_list.append(inst.exchange)
  abbrevSymbol_list.append(inst.abbrev_symbol)
  shenwanIndustryCode_list.append(inst.shenwan_industry_code)
  shenwanIndustryName_list.append(inst.shenwan_industry_name)
  industryCode_list.append(inst.industry_code)
  industryName_list.append(inst.industry_name)
  sectorCode_list.append(inst.sector_code)
  sectorName_list.append(inst.sector_code_name)
  
instrument_df = pd.DataFrame({"bookId":bookId_list,
                             "exchange":exchange_list,
                             "abbrevSymbol":abbrevSymbol_list,
                             "shenwanIndustryCode":shenwanIndustryCode_list,
                             "shenwanIndustryName":shenwanIndustryName_list,
                             "industryCode":industryCode_list,
                             "industryName":industryName_list,
                             "sectorCode":sectorCode_list,
                             "sectorName":sectorName_list})

In [29]:
instrument_df.to_csv("cn_instrument_info_{}_{}.csv".format(year_start,year_end))

In [7]:
fundamental_data = {}
query_dates = trade_dates[(trade_dates >= date_start_dt) & (trade_dates <= date_end_dt)]
ndates = len(query_dates)
for counter,dt in enumerate(query_dates):
    print("Date: {} | Progress: {}%".format(dt,(counter+1)/ndates*100))
    _fundamental_data = get_fundamentals(
        query(
        fundamentals.eod_derivative_indicator.market_cap, #总市值
        fundamentals.eod_derivative_indicator.a_share_market_val_2, #流通市值
        fundamentals.cash_flow_statement.cash_received_from_sales_of_goods, #销售额 - 单季/同比
        fundamentals.eod_derivative_indicator.pb_ratio, #净资产/总市值=市净率
        fundamentals.income_statement.net_profit, #净利润
        fundamentals.eod_derivative_indicator.ps_ratio #市销率
        )
      .filter(fundamentals.income_statement.stockcode.in_(stock_list))
      , 
        entry_date=dt, interval='1q', report_quarter=True
    )
    _fundamental_data = _fundamental_data.to_frame()
    _fundamental_data.index.names = ['date', 'order_book_id']
    fundamental_data[dt] = _fundamental_data
    
fundamental_data = pd.concat(fundamental_data)
fundamental_data.reset_index(level=0, drop=True, inplace=True)

Date: 2012-01-04 00:00:00 | Progress: 0.05875440658049354%
Date: 2012-01-05 00:00:00 | Progress: 0.11750881316098707%
Date: 2012-01-06 00:00:00 | Progress: 0.17626321974148063%
Date: 2012-01-09 00:00:00 | Progress: 0.23501762632197415%
Date: 2012-01-10 00:00:00 | Progress: 0.2937720329024677%
Date: 2012-01-11 00:00:00 | Progress: 0.35252643948296125%
Date: 2012-01-12 00:00:00 | Progress: 0.4112808460634548%
Date: 2012-01-13 00:00:00 | Progress: 0.4700352526439483%
Date: 2012-01-16 00:00:00 | Progress: 0.5287896592244419%
Date: 2012-01-17 00:00:00 | Progress: 0.5875440658049353%
Date: 2012-01-18 00:00:00 | Progress: 0.6462984723854289%
Date: 2012-01-19 00:00:00 | Progress: 0.7050528789659225%
Date: 2012-01-20 00:00:00 | Progress: 0.763807285546416%
Date: 2012-01-30 00:00:00 | Progress: 0.8225616921269095%
Date: 2012-01-31 00:00:00 | Progress: 0.881316098707403%
Date: 2012-02-01 00:00:00 | Progress: 0.9400705052878966%


KeyboardInterrupt: 

In [ ]:
trade_ts = trade_data.to_frame()
trade_ts.index.names = ['date', 'order_book_id']

return_ts = pd.DataFrame(return_data.stack(), columns=['return'])
return_ts.index.names = ['date', 'order_book_id']

turnover_ts = turnover_data.to_frame()
turnover_ts.index.names = ['date', 'order_book_id']

data = return_ts.merge(trade_ts, how='left', left_index=True, right_index=True)
data = data.merge(turnover_ts, how='left', left_index=True, right_index=True)
data = data.merge(fundamental_data, how='left', left_index=True, right_index=True)

In [ ]:
data.head()

In [ ]:
# Save Data
data.to_csv("cn_equity_daily_{}_{}.csv".format(year_start,year_end))